# Engineering of dataset for model training

The code engineers a dataset and samples of earthquake events in two specific geographic subregions. The events are represented in a set of dataframes, while the contextual data related to each event (such as environmental data) is represented in an xarray dataset.

In the first section of the code, the earthquake event data is filtered to include only those events that occurred at least one week after a given minimum date. This is done to ensure that sufficient contextual data is available for each event.

The function select_data_for_event extracts the date and location (latitude and longitude) of an earthquake event from the event data. It then selects the corresponding data from the xarray dataset, finding the nearest data point if the exact location is not available.

The function select_grid_around_event creates a square grid around the location of an earthquake event. The grid extends by a specified size in latitude and longitude from the event location. The function also defines a time period, starting a specified number of days before the event and ending the day before the event. It then selects all data within this grid and time period from the xarray dataset, returning the data as a 4D numpy array along with the corresponding arrays of latitudes, longitudes, and dates.

The function select_random_location is used to generate negative samples for the analysis. It randomly selects a location and date, making sure the location is under the sea and the date is within the range of the event data. It then checks whether there were any earthquake events within a grid around this location and within a time period around this date. If there were no events, the function selects the contextual data for this location and date as a negative sample, using the same method as select_grid_around_event.

The functions generate_samples, generate_negative_samples, and combine_samples_and_labels are used to generate positive and negative samples for the analysis and combine them into a single array of samples and an array of corresponding labels. The positive samples are generated by applying select_grid_around_event to each event in the event data, and the negative samples are generated by applying select_random_location a number of times equal to the number of positive samples.

In the final part of the script, the functions are applied to the event data and xarray datasets for the two subregions. The resulting samples and labels for each subregion are then combined into a single array of samples and labels for the entire analysis.

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString, MultiLineString, box
from scipy.spatial import cKDTree
import folium
from folium import Rectangle
import xarray as xr
import random
import numpy as np


In [ ]:
MIN_DATE = '2013-01-01'
MAX_DATE = '2023-06-01'
BATHYMETRY_FILE = '../gebco_2022_n45.864_s29.918_w-5.947_e36.261.nc'
# FAULTS_FILE = '../fault_data.nc'
FAULTS_FILE = '../AFEAD_v2022.shp'

grid_size = 0.5  # the spatial periphery of each sample point in degrees
days_before = 7  # the temporal window before each sample point in days
# MIN_DATE_OFFSET = pd.DateOffset(days=days_before)

variables = ['CHL', 'analysed_sst', 'KD490', 'RRS555', 'BBP', 'elevation', 'fault_distance', 'fault_CONF'] # for full dataset
# variables = ['CHL', 'analysed_sst'] # for testing

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# list for storing sample points
sample_events = []

SUBREGIONS = [
    {
        'name': 'subregion1',
        'quake_latmin': 38.2,
        'quake_latmax': 40.6,
        'quake_lonmin': 24.4,
        'quake_lonmax': 26.8,
        'latmin': 38.2 - grid_size,
        'latmax': 40.6 + grid_size,
        'lonmin': 24.4 - grid_size,
        'lonmax': 26.8 + grid_size,
        'csv_file': 'subregion1.csv',
        'earthquake_data': None,
    },
    {
        'name': 'subregion2',
        'quake_latmin': 35.8,
        'quake_latmax': 38.2,
        'quake_lonmin': 23.9,
        'quake_lonmax': 26.3,
        'latmin': 35.8 - grid_size,
        'latmax': 38.2 + grid_size,
        'lonmin': 23.9 - grid_size,
        'lonmax': 26.3 + grid_size,
        'csv_file': 'Subregion2_fixed.csv',
        'earthquake_data': None,
    },
]

COUNTRY_BORDERS = [
    'gadm41_GRC_3.json',
    'gadm41_TUR_2.json'
]


In [ ]:
# fault line preparation functions

# Function to simplify the geometry of GeoDataFrame 
def simplify_geometry(gdf, tolerance):
    # Simplify LineStrings, keeping the topological validity of geometries intact
    # The tolerance parameter specifies the maximum allowable offset during simplification
    gdf['geometry'] = gdf['geometry'].simplify(tolerance)
    return gdf

# Function to filter the GeoDataFrame by the bounding box of the dataset
def filter_by_bounding_box(gdf, ds):
    # Define the bounding box (bbox) coordinates based on the min and max lat-lon values from the dataset
    latmin = ds.lat.min().values
    latmax = ds.lat.max().values
    lonmin = ds.lon.min().values
    lonmax = ds.lon.max().values

    # Create the bbox as a GeoDataFrame
    bbox = gpd.GeoDataFrame(geometry=[box(lonmin, latmin, lonmax, latmax)], crs=gdf.crs)

    # Perform spatial join between the bbox and the gdf to retain only the geometries that intersect with the bbox
    gdf = gpd.sjoin(gdf, bbox, op='intersects')
    # Drop the index column added by the spatial join operation
    gdf = gdf.drop(columns='index_right')

    return gdf

# Function to convert geometries into individual lat-lon points
def get_fault_coordinates(gdf):
    rows_list = []
    for index, row in gdf.iterrows():
        geom = row['geometry']
        # If the geometry is LineString, get its coordinates directly
        if isinstance(geom, LineString):
            coords = geom.coords
        # If the geometry is MultiLineString, get coordinates from each LineString it consists of
        elif isinstance(geom, MultiLineString):
            coords = [pt for line in geom.geoms for pt in line.coords]
        else:
            raise ValueError(f"Unknown geometry type: {geom.type}")
        
        # For each coordinate point, create a new row with the original row's data and the point's lat-lon values
        for point in coords:
            dict1 = {}
            dict1.update(row)  # get other columns
            dict1.update({'lon': point[0], 'lat': point[1]})
            rows_list.append(dict1)

    # Create new DataFrame from the list of new rows and drop old geometry column
    gdf_expanded = pd.DataFrame(rows_list)
    gdf_expanded = gdf_expanded.drop(columns=['geometry'])

    # Return the lat-lon coordinates of faults and the expanded DataFrame
    return gdf_expanded[['lat', 'lon']].values, gdf_expanded

# Function to create a KDTree from fault coordinates and query it for each point in the dataset
def create_kdtree_and_query(ds_coords, fault_coords):
    # Build a KDTree from fault_coords for efficient spatial queries
    tree = cKDTree(fault_coords)

    # Query the KDTree to find the closest fault to each point in the dataset
    distances, indices = tree.query(ds_coords)

    # # Store the shape of the original lat-lon data
    # ds_shape = ds_coords.reshape(-1, 2).shape

    return distances, indices

# Main function to add fault information to the dataset
def add_faults_to_dataset(ds, shapefile, tolerance=0.01):
    # Read in the shapefile
    gdf = gpd.read_file(shapefile)

    # Simplify geometry and filter by bounding box
    gdf = simplify_geometry(gdf, tolerance)
    gdf = filter_by_bounding_box(gdf, ds)

    # Get fault coordinates
    fault_coords, gdf_expanded = get_fault_coordinates(gdf)

    # Get all coordinates from the dataset
    lats, lons = np.meshgrid(ds.lat.values, ds.lon.values)
    ds_shape = lats.shape  # Store the shape of the original lat-lon data
    ds_coords = np.dstack([lats, lons]).reshape(-1, 2)
    
    # Perform KDTree query
    distances, indices = create_kdtree_and_query(ds_coords, fault_coords)

    # Reshape the data to have the same 2D structure as the original lat-lon data
    distances = distances.reshape(ds_shape)
    indices = indices.reshape(ds_shape)

    # Create DataArrays for distance and CONF
    ds['fault_distance'] = xr.DataArray(distances, coords=[ds.lon, ds.lat], dims=['lon', 'lat'])
    ds['fault_CONF'] = xr.DataArray(gdf_expanded.CONF.values[indices], coords=[ds.lon, ds.lat], dims=['lon', 'lat'])

    return ds


In [ ]:
# Country polygon data for the region is used to create a land mask
def get_polygons(countries):
    polygons = []
    for country in countries:
        data = gpd.read_file(country)
        polygons.extend(data['geometry'])
    return polygons

# The land mask is used to remove earthquakes that occur on land
def is_earthquake_on_land(point, polygons):
    return any(point.within(polygon) for polygon in polygons)

# Function to identify points that are on land
def is_coordinate_offshore(lat, lon, polygons):
    point = Point(lon, lat)
    return not any(point.within(polygon) for polygon in polygons)

# Function to remove earthquakes that occur on land
def remove_land_earthquakes(df, polygons):
    # print columns
    print(df.columns)
    df['point'] = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    df = df[~df['point'].apply(is_earthquake_on_land, args=(polygons,))]
    return df.drop(columns='point')

def add_earthquake_markers(m, df):
    for _, row in df.iterrows():
        popup = 'time: {time}<br>latitude: {latitude}<br>longitude: {longitude}<br>depth: {depth}'.format(**row)
        folium.Marker([row['latitude'], row['longitude']], popup=popup).add_to(m)
    return m

# Function to add subregion borders to the map
def add_subregion_border(m, subregion):
    bounds = [[subregion['quake_latmin'], subregion['quake_lonmin']], [subregion['quake_latmax'], subregion['quake_lonmax']]]
    Rectangle(bounds, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(m)


# Function to load data files
def load_data_files(subregion_name):
    chl_file = f'OCEANCOLOUR_MED_BGC_L4_MY_009_144/colour_CHL_{subregion_name}_L.nc'
    sst_file1 = f'SST_MED_SST_L4_NRT_OBSERVATIONS_010_004/sst__{subregion_name}_L.nc'
    sst_file2 = f'SST_MED_SST_L4_NRT_OBSERVATIONS_010_004/sst__{subregion_name}_L2.nc'
    optics_file = f'OCEANCOLOUR_MED_BGC_L3_MY_009_143/optics_BBP443_{subregion_name}_L_refined.nc'
    refl_file = f'OCEANCOLOUR_MED_BGC_L3_MY_009_143/refl_{subregion_name}_L_refined.nc'
    transp_file = f'OCEANCOLOUR_MED_BGC_L3_MY_009_143/transparency_{subregion_name}_L_refined.nc'

    chl = xr.open_dataset(chl_file)
    sst = xr.concat([xr.open_dataset(sst_file1), xr.open_dataset(sst_file2)], dim='time')
    optics = xr.open_dataset(optics_file)
    refl = xr.open_dataset(refl_file)
    transp = xr.open_dataset(transp_file)
    
    return chl, sst, optics, refl, transp

# Function to merge datasets with spatial resolution and geographical area adjustment
def merge_datasets(chl, sst, optics, refl, transp, subregion):
    # regrid the SST dataset to match the spatial resolution and geographical area of the chlorophyll dataset
    print('Regridding SST dataset...')
    sst = sst.interp(lat=chl.lat, lon=chl.lon, method='linear')
    print('Regridding optics dataset...')
    optics = optics.interp(lat=chl.lat, lon=chl.lon, method='linear')
    print('Regridding refl dataset...')
    refl = refl.interp(lat=chl.lat, lon=chl.lon, method='linear')
    print('Regridding transp dataset...')
    transp = transp.interp(lat=chl.lat, lon=chl.lon, method='linear')
    
    print('Merging datasets...')
    # merge CHL and SST data
    ds = xr.merge([chl, sst, optics, refl, transp])

    # load Bathymetry data
    ds_bathy = xr.open_dataset(BATHYMETRY_FILE)
    # select geographical boundaries
    ds_bathy = ds_bathy.sel(lat=slice(subregion['latmin'], subregion['latmax']), 
                            lon=slice(subregion['lonmin'], subregion['lonmax']))
    # regrid the bathymetry dataset to match the spatial resolution and geographical area of the chlorophyll dataset
    print('Regridding bathymetry dataset...')
    ds_bathy = ds_bathy.interp(lat=ds.lat, lon=ds.lon, method='linear')

    print('Merging datasets...')
    # merge bathymetry data with CHL and SST data
    ds = xr.merge([ds, ds_bathy])

    # load Fault line data
    # ds_faults = xr.open_dataset(FAULTS_FILE)
    # # select geographical boundaries
    # ds_faults = ds_faults.sel(lat=slice(subregion['latmin'], subregion['latmax']), 
    #                           lon=slice(subregion['lonmin'], subregion['lonmax']))
    # # merge fault line data with CHL, SST, and bathymetry data
    # ds = xr.merge([ds, ds_faults])

    # load and process fault line data
    print('Processing fault line data...')
    ds = add_faults_to_dataset(ds, FAULTS_FILE, tolerance=0.0001)

    return ds

# Handle missing values
def handle_missing_values(subregion):
    print('Handling missing values...')
    subregion['CHL'] = subregion['CHL'].fillna(-99)
    subregion['BBP'] = subregion['BBP'].fillna(-99)
    subregion['KD490'] = subregion['KD490'].fillna(-99)
    subregion['RRS555'] = subregion['RRS555'].fillna(-99)
    subregion['analysed_sst'] = subregion['analysed_sst'].fillna(-99)
    subregion['elevation'] = subregion['elevation'].fillna(subregion['elevation'].mean())
    subregion['fault_distance'] = subregion['fault_distance'].fillna(subregion['fault_distance'].mean())
    subregion['fault_CONF'] = subregion['fault_CONF'].fillna('0')
    # subregion['fault_RATE'] = subregion['fault_RATE'].fillna('0')

    return subregion

# Convert CONF and RATE to numeric values
def convert_rate_conf(subregion):
    conf_mapping = {'A': 4, 'B': 3, 'C': 2, 'D': 1, '0': 0, '': 0}
    rate_mapping = {'3': 3, '2': 2, '1': 1, '0': 0, '': 0}
    
    subregion['fault_CONF'] = xr.DataArray(pd.Series(subregion['fault_CONF'].values.flatten()).map(conf_mapping).values.reshape(subregion['fault_CONF'].shape), 
                                     coords=subregion['fault_CONF'].coords, 
                                     dims=subregion['fault_CONF'].dims)
    
    # subregion['fault_RATE'] = xr.DataArray(pd.Series(subregion['fault_RATE'].values.flatten()).map(rate_mapping).values.reshape(subregion['fault_RATE'].shape),
    #                                     coords=subregion['fault_RATE'].coords,
    #                                     dims=subregion['fault_RATE'].dims)
    
    return subregion

# Function to format and handle earthquake data
def handle_earthquake_data(earthquakes_subregion):
    print('Handling earthquake data...')
    earthquakes_subregion['date'] = earthquakes_subregion['time'].astype(str).str[:10].astype('datetime64[ns]')
    return earthquakes_subregion

def process_subregion(subregion, earthquakes_subregion):
    chl, sst, optics, refl, transp = load_data_files(subregion['name'])
    ds = merge_datasets(chl, sst, optics, refl, transp, subregion)
    ds.attrs = {}
    ds = handle_missing_values(ds)
    ds = convert_rate_conf(ds)
    earthquakes_subregion = handle_earthquake_data(earthquakes_subregion)
    return ds, earthquakes_subregion


def filter_events_by_date(earthquakes, min_date):
    return earthquakes[earthquakes['date'] > pd.to_datetime(min_date) + pd.DateOffset(days=days_before)]



def select_data_for_event(ds, event):
    # Extract event details
    date = pd.to_datetime(event['date'])
    lat = event['latitude']
    lon = event['longitude']

    # Select nearest lat/lon first
    ds_nearest = ds.sel(lat=lat, lon=lon, method='nearest')

    # Extract nearest lat/lon
    nearest_lat = ds_nearest.lat.values
    nearest_lon = ds_nearest.lon.values

    # add to samples_events
    sample_events.append([date, nearest_lat, nearest_lon])

    return nearest_lat, nearest_lon, date

def select_grid_around_event(ds, event, grid_size, days_before, variables):
    # Extract event details
    lat, lon, date = select_data_for_event(ds, event)

    # Define grid boundaries
    lat_min = lat - grid_size
    lat_max = lat + grid_size
    lon_min = lon - grid_size
    lon_max = lon + grid_size

    # Define date range
    date_min = date - pd.DateOffset(days=days_before)
    date_max = date - pd.DateOffset(days=1)  # Exclude the day of the event itself

    # Select data within grid and date range
    ds_grid = ds.sel(
        lat=slice(lat_min, lat_max),
        lon=slice(lon_min, lon_max),
        time=slice(date_min, date_max),
    )

    # Select variables
    ds_grid = ds_grid[variables]

    # Get arrays of latitudes, longitudes, and dates
    lats = ds_grid.lat.values
    lons = ds_grid.lon.values
    times = ds_grid.time.values

    # Get array of data values
    values = ds_grid.to_array().transpose('time', 'lat', 'lon', 'variable').values



    return values, lats, lons, times

def select_random_location(ds, df_events, grid_size, days_before):
    # Extract lat and lon values from the dataset
    lat_values = ds.lat.values
    lon_values = ds.lon.values

    # min(lat_value)+0.5 <= lat_values <= max(lat_values)-0.5
    # min(lon_value)+0.5 <= lon_values <= max(lon_values)-0.5
    lat_values = lat_values[(lat_values >= lat_values.min() + grid_size) & (lat_values <= lat_values.max() - grid_size)]
    lon_values = lon_values[(lon_values >= lon_values.min() + grid_size) & (lon_values <= lon_values.max() - grid_size)]

    # Extract the date range from df_events
    date_range = [df_events['date'].min(), df_events['date'].max()]

    while True:
        # Select a random latitude and longitude within the given ranges
        lat = random.choice(lat_values)
        lon = random.choice(lon_values)
        date = random.choice(pd.date_range(*date_range))

        # Define grid boundaries and date range
        lat_min, lat_max = lat - grid_size, lat + grid_size
        lon_min, lon_max = lon - grid_size, lon + grid_size
        date_min = date - pd.DateOffset(days=days_before)
        date_max = date - pd.DateOffset(days=1)

        # Check if there is any earthquake event within the grid and date range
        event_in_range = ((df_events['latitude'].between(lat_min, lat_max)) & 
                          (df_events['longitude'].between(lon_min, lon_max)) & 
                          (df_events['date'].between(date_min, date_max))).any()

        # Check if lat and lon occurs at sea using is_coordinate_offshore()
        lat_lon_offshore = is_coordinate_offshore(lat, lon, polygons=polygons)

        if not event_in_range and lat_lon_offshore:
            return select_grid_around_event(ds, {'date': date, 'latitude': lat, 'longitude': lon}, grid_size, days_before, variables)

def generate_samples(ds, df_events, grid_size, days_before):
    return [select_grid_around_event(ds, event, grid_size, days_before, variables) for event in df_events.to_dict('records')]

def generate_negative_samples(ds, df_events, num_samples, grid_size, days_before):
    return [select_random_location(ds, df_events, grid_size, days_before) for _ in range(num_samples)]

def combine_samples_and_labels(positive_samples, negative_samples):
    num_positive = len(positive_samples)
    num_negative = len(negative_samples)

    samples = positive_samples + negative_samples
    labels = [1] * num_positive + [0] * num_negative

    return samples, labels

def process_samples_and_labels(ds, earthquakes, min_date, grid_size, days_before):
    df_events = filter_events_by_date(earthquakes, min_date)

    # Generate positive samples for earthquakes in the region
    positive_samples = generate_samples(ds, df_events, grid_size, days_before)
    print(f"Positive samples in this region: {len(positive_samples)}")

    # Generate an equal number of negative samples for non-earthquakes
    negative_samples = generate_negative_samples(ds, df_events, len(positive_samples), grid_size, days_before)
    print(f"Negative samples in this region: {len(negative_samples)}")

    # raise error if there is shape inconsistency across all samples
    if not all([positive_samples[0][0].shape == sample[0].shape for sample in positive_samples]):
        raise ValueError("Inconsistent shape across samples")
    
    # Print the shape of the data arrays
    if positive_samples:
        print(f"Shape of data arrays: {positive_samples[0][0].shape}")
    
    return combine_samples_and_labels(positive_samples, negative_samples)


In [ ]:
polygons = get_polygons(COUNTRY_BORDERS)

m = folium.Map(location=[39.5, 25.5], zoom_start=6) # Initial map focused on the Aegean Sea

for region in SUBREGIONS:
    df = pd.read_csv(region['csv_file'])
    # print(f"Data from {region['csv_file']}:")
    # print(df.head())  # Prints first 5 rows of the dataframe
    if 'longitude' in df.columns:
        df = remove_land_earthquakes(df, polygons)
        region['earthquake_data'] = df
        m = add_earthquake_markers(m, df)
        add_subregion_border(m, region)
    else:
        print(f"'longitude' column not found in {region['csv_file']}")
    
display(m)

In [ ]:
# Process the data for each subregion
ds_subregion1, earthquakes_subregion1 = process_subregion(SUBREGIONS[0], SUBREGIONS[0]['earthquake_data'])
ds_subregion2, earthquakes_subregion2 = process_subregion(SUBREGIONS[1], SUBREGIONS[1]['earthquake_data'])


In [ ]:
# Generate samples and labels for each subregion
samples_subregion1, labels_subregion1 = process_samples_and_labels(ds_subregion1, earthquakes_subregion1, MIN_DATE, grid_size, days_before)
samples_subregion2, labels_subregion2 = process_samples_and_labels(ds_subregion2, earthquakes_subregion2, MIN_DATE, grid_size, days_before)

# Combine samples and labels for all subregions
samples = samples_subregion1 + samples_subregion2
labels = labels_subregion1 + labels_subregion2

# Convert the lists to numpy arrays
X = np.array(samples)
y = np.array(labels)

In [ ]:
# Describe the variables in the dataset
print('Variables in subregion 1:', ds_subregion1.data_vars)
print('Variables in subregion 2:', ds_subregion2.data_vars)